<a href="https://colab.research.google.com/github/PeterRoumeliotis/AIFireSmokeDetectionResearchProject/blob/main/AIFireAndSmokeDetectionResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import numpy as np
import time
from google.colab import files

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phylake1337/fire-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fire-dataset


In [3]:
random.seed(42)

# Finding source paths
source_dir = '/kaggle/input/fire-dataset/fire_dataset'
source_fire_dir = os.path.join(source_dir, 'fire_images')
source_non_fire_dir = os.path.join(source_dir, 'non_fire_images')

# Train and validation paths
dest_dir = 'data'
train_fire_dir = os.path.join(dest_dir, 'train', 'fire')
val_fire_dir = os.path.join(dest_dir, 'validation', 'fire')
train_non_fire_dir = os.path.join(dest_dir, 'train', 'non_fire')
val_non_fire_dir = os.path.join(dest_dir, 'validation', 'non_fire')

# Creating the directories if they don't exist
os.makedirs(train_fire_dir, exist_ok=True)
os.makedirs(val_fire_dir, exist_ok=True)
os.makedirs(train_non_fire_dir, exist_ok=True)
os.makedirs(val_non_fire_dir, exist_ok=True)

# 80% training, 20% validation
split_ratio = 0.8

# Splitting the files into each folder
def split_data(source_folder, train_folder, val_folder, split_ratio=0.8):

    file_list = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    random.shuffle(file_list)

    split_point = int(len(file_list) * split_ratio)
    train_files = file_list[:split_point]
    val_files = file_list[split_point:]

    for file_name in train_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(train_folder, file_name)
        shutil.copy(src, dst)

    for file_name in val_files:
        src = os.path.join(source_folder, file_name)
        dst = os.path.join(val_folder, file_name)
        shutil.copy(src, dst)

split_data(source_fire_dir, train_fire_dir, val_fire_dir, split_ratio)
split_data(source_non_fire_dir, train_non_fire_dir, val_non_fire_dir, split_ratio)

print("Data has been successfully split into training and validation")

Data has been successfully split into training and validation


In [4]:
img_width, img_height = 150, 150

batch_size = 32
epochs = 20

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

# Augmenting training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Generate validation batches
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 799 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


# My Model

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

model_save_path = "fire_detection_cnn.keras"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 30s 969ms/step - accuracy: 0.7910 - loss: 0.6623 - val_accuracy: 0.9323 - val_loss: 0.2250
Epoch 2/20
 1/24 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9062 - loss: 0.3613

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.9062 - loss: 0.3613 - val_accuracy: 0.9271 - val_loss: 0.2307
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 891ms/step - accuracy: 0.9117 - loss: 0.2717 - val_accuracy: 0.9115 - val_loss: 0.2362
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 179ms/step - accuracy: 0.9677 - loss: 0.1045 - val_accuracy: 0.8854 - val_loss: 0.3324
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 35s 831ms/step - accuracy: 0.9269 - loss: 0.1777 - val_accuracy: 0.9375 - val_loss: 0.1315
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.9355 - loss: 0.1097 - val_accuracy: 0.9271 - val_loss: 0.1531
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 20s 847ms/step - accuracy: 0.9623 - loss: 0.1130 - val_accuracy: 0.9583 - val_loss: 0.0723
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 1.0000 - loss: 0.0522 - val_accuracy: 0.9583 - val_loss: 0.0765
Epoch 9/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 20s 864ms/step - accuracy: 0.9449 - loss: 0.1366 - val_accuracy: 0.9635 - 

In [7]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00


In [8]:
import os

for split in ["train","validation"]:
    for cls_idx, cls in enumerate(["non_fire","fire"]):
        folder = f"data/{split}/{cls}"
        for img in os.listdir(folder):
            if not img.lower().endswith((".jpg",".png")): continue
            img_path   = os.path.join(folder, img)
            label_path = os.path.splitext(img_path)[0] + ".txt"

            if cls == "fire":
                with open(label_path, "w") as f:
                    f.write(f"{1} 0.5 0.5 1.0 1.0\n")
            else:
                open(label_path, "w").close()


In [9]:
%%bash
cat <<EOF > data/fire_data.yaml
train: /content/data/train
val:   /content/data/validation

names:
  0: non_fire
  1: fire
EOF

echo "Created YAML:"
cat data/fire_data.yaml


Created YAML:
train: /content/data/train
val:   /content/data/validation

names:
  0: non_fire
  1: fire


In [11]:
from ultralytics import YOLO

yolo = YOLO('yolov8n.pt')

yolo.train(
    data='data/fire_data.yaml',
    epochs=20,
    imgsz=150,
    batch=32,
    project='yolo-fire',
    name='exp'
)

yolo_model = YOLO('yolo-fire/exp/weights/best.pt')


Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/fire_data.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=150, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolo-fire, name=exp2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=Non

train: Scanning /content/data/train/fire.cache... 798 images, 194 backgrounds, 1 corrupt: 100%|██████████| 799/799 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/data/train/fire/fire.357.png: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/data/train/fire/fire.576.png: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/data/train/fire/fire.681.png: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/data/train/non_fire/non_fire.189.png: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'heic', 'jpg', 'bmp', 'dng', 'pfm', 'tiff', 'tif', 'mpo', 'png', 'jpeg', 'webp'}
videos: {'mpeg', 'mkv', 'ts', 'mov', 'mpg', 'webm', 'avi', 'asf', 'gif', 'wmv', 'mp4', 'm4v'}
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 368.1±98.4 MB/s, size: 299.9 KB)


val: Scanning /content/data/validation/fire.cache... 200 images, 49 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


...

      20/20     0.533G     0.0932     0.1746     0.8476         27        160: 100%|██████████| 25/25 [00:10<00:00,  2.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.17it/s]

                   all        200        151      0.979      0.993      0.994      0.994



20 epochs completed in 0.085 hours.
Optimizer stripped from yolo-fire/exp2/weights/last.pt, 6.2MB
Optimizer stripped from yolo-fire/exp2/weights/best.pt, 6.2MB

Validating yolo-fire/exp2/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.91it/s]


                   all        200        151      0.993      0.986      0.995      0.995
                  fire        151        151      0.993      0.986      0.995      0.995
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to yolo-fire/exp2


In [12]:
metrics = yolo_model.val()
print(metrics)

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1759.4±1253.7 MB/s, size: 122.2 KB)


val: Scanning /content/data/validation/fire.cache... 200 images, 49 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.27it/s]


                   all        200        151      0.993      0.986      0.995      0.995
                  fire        151        151      0.993      0.986      0.995      0.995
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aa9b42adc10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026, 

In [13]:
uploaded = files.upload()
video_path = next(iter(uploaded.keys()))
print("Video file:", video_path)

cnn_model = tf.keras.models.load_model('fire_detection_cnn.keras')

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
cnn_times, yolo_times = [], []
cnn_detections, yolo_detections = [], []

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    timestamp = frame_idx / fps

    # CNN
    t0 = time.perf_counter()
    img = cv2.resize(frame, (150,150)) / 255.0
    p = cnn_model.predict(np.expand_dims(img,0), verbose=0)[0][0]
    t1 = time.perf_counter()
    cnn_times.append(t1 - t0)
    if p > 0.5 and not cnn_detections:
        cnn_detections.append(timestamp)

    # YOLOv8
    t2 = time.perf_counter()
    results = yolo_model(frame, verbose=False)
    t3 = time.perf_counter()
    yolo_times.append(t3 - t2)
    for r in results:
        if r.boxes.cls.cpu().numpy().tolist().count(1) > 0 and not yolo_detections:
            yolo_detections.append(timestamp)
            break

cap.release()

import numpy as _np
print(f"CNN first detection at {cnn_detections or ['never']} seconds")
print(f"YOLO first detection at {yolo_detections or ['never']} seconds")
print(f"Avg CNN frame time: {_np.mean(cnn_times):.4f}s")
print(f"Avg YOLO frame time: {_np.mean(yolo_times):.4f}s")


Saving videoplayback.mp4 to videoplayback.mp4
Video file: videoplayback.mp4
CNN first detection at [0.03336666666666667] seconds
YOLO first detection at [0.03336666666666667] seconds
Avg CNN frame time: 0.0758s
Avg YOLO frame time: 0.0106s
